In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../')
from modules import utils, constants
from modules.env import LupusEnv
import stable_baselines
from stable_baselines.common.vec_env import DummyVecEnv

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWa

In [2]:
SEED = 84

In [3]:
def load_model(filepath, model_type, env):
    if model_type.lower() == 'dqn3': #dqn3
        return stable_baselines3.DQN.load(filepath, env=env)
    elif model_type.lower() == 'dqn': #dqn
        return stable_baselines.DQN.load(filepath, env=env)
    elif model_type.lower() == 'ppo': #PPO
        return stable_baselines3.PPO.load(filepath, env=env)
    elif model_type.lower() == 'ppo2': #PPO2
        return stable_baselines.PPO2.load(filepath, env=env)
    elif model_type.lower() == 'ppo3': #PPO3
        return stable_baselines3.PPO3.load(filepath, env=env) 
    elif model_type.lower() == 'ac': #AC
        return stable_baselines3.AC.load(filepath, env=env) 
    elif model_type.lower() == 'a2c': #A2C
#         return stable_baselines3.A2C.load(filepath, env=env)
        return stable_baselines.A2C.load(filepath, env=env)
    elif model_type.lower() == 'acer': #ACER
        return stable_baselines.ACER.load(filepath, env=env) 
    elif model_type.lower() == 'ddpg': #DDPG
        return stable_baselines3.DDPG.load(filepath, env=env)
    elif model_type.lower() == 'pg': #PG
        return stable_baselines3.PG.load(filepath, env=env)
    elif model_type.lower() == 'acktr': #TD3
        return stable_baselines.ACKTR.load(filepath, env=env)
    elif model_type.lower() == 'trpo': #TRPO
        return sb3_contrib.TRPO.load(filepath, env=env)
    else:
        raise ValueError(f'Unknown model type {model_type}')

In [4]:
from gym.spaces import Box
def create_test_df(model_path, model_type, X_test, y_test):
    testing_env = DummyVecEnv([lambda: LupusEnv(X_test, y_test, random=False)])
#     testing_env = LupusEnv(X_test, y_test, random=False)
    testing_env.seed(SEED)
    testing_env.observation_space = Box(0, 3, (constants.FEATURE_NUM,))
    model = load_model(model_path, model_type, testing_env)
    test_df = pd.DataFrame()
    try:
        while True:
            obs, done = testing_env.reset(), False
            while not done:
                action, states = model.predict(obs, deterministic=True)
                obs, rew, done, info = testing_env.step(action)
                if done==True:
                    test_df = test_df.append(info, ignore_index=True)

    except StopIteration:
        pass
    min_path_length = test_df.episode_length.min()
    average_path_length = test_df.episode_length.mean()
    max_path_length = test_df.episode_length.max()
    return test_df

In [5]:
testing_df = pd.read_csv('../data/missingness/0/testing_set.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [6]:
X_test = testing_df.iloc[:, 0:-1]
y_test = testing_df.iloc[:, -1]
X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((14000, 23), (14000,))

In [7]:
# test_df = create_test_df('../models/logs/sb/dueling_ddqn/best_acc_model.zip', 'dqn', X_test, y_test)
test_df = create_test_df('../models/logs/sb/dqn_seed_84/best_acc_model.zip', 'dqn', X_test, y_test)
test_df.head()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.


,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success,terminal_observation
0,0,17,0.466667,1,1,"[anti_cardioliphin_antibodies, anti_β2gp1_anti...",False,True,"[-1.0, 0.0, 0.0, -1.0, -1.0, 0.0, -1.0, -1.0, ..."
1,2,19,0.400000,1,1,"[anti_cardioliphin_antibodies, anti_β2gp1_anti...",False,True,"[-1.0, 0.0, 0.0, -1.0, 0.0, 0.0, -1.0, -1.0, 0..."
2,4,17,0.466667,1,1,"[anti_cardioliphin_antibodies, anti_β2gp1_anti...",False,True,"[1.0, -1.0, 0.0, -1.0, 0.0, 0.0, -1.0, -1.0, 1..."
3,6,20,-1.633333,1,0,"[anti_cardioliphin_antibodies, anti_β2gp1_anti...",False,False,"[1.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0..."
4,8,18,0.433333,1,1,"[anti_cardioliphin_antibodies, anti_β2gp1_anti...",False,True,"[1.0, -1.0, 0.0, 0.0, -1.0, 1.0, -1.0, -1.0, 1..."


In [8]:
acc, f1, roc_auc = utils.test(test_df['y_actual'], test_df['y_pred'])
acc, f1, roc_auc

(85.05714285714285, 85.48424773819033, 85.59729881535947)

In [9]:
test_df.episode_length.min(), test_df.episode_length.mean(), test_df.episode_length.max()

(12, 18.199, 21)

In [10]:
test_df.to_csv('../test_dfs/23_03_23/vanilla_dqn_seed84_basic_test_df.csv', index=False)